# What3D
分类任务，支持两种模式
1. Folder模式，需要输入`train`, `valid`两个测试集对应的目录。`labels.txt`，需要训练的label，里面每个类别一行。
2. List模式，需要输入`train`, `valid`两个测试集对应的训练文件，每行一个样本。`labels.txt`是可选参数，里面每个类别一行。`data_pattern`一个通用的目录，与train、val中的第一列进行拼接。

### 支持的模型名称

模型名称替换代码中的 `model_name`变量的值。

| **模型系列** | **模型名称**                                                 |
| ------------ | ------------------------------------------------------------ |
| ResNet       | resnet10, resnet18, resnet34, resnet50, resnet101, resnet152, resnet200 |
| DenseNet     | DenseNet121, DenseNet201, DenseNet169, DenseNet264           |
| ShuffleNet   | ShuffleNet |
| VisionTransformer     | ViT, SimpleViT, CCT         |
| OnekeyAI | UNetClassification |     

### List模式

在Onekey中List模式一般是采用labelme标注出来的结果，如果要使用自己的数据应用List模式，需要根据自己的实际情况对数据进行处理。

* `train.txt`，训练数据列表，中间用\t（Tab水平制表符）进行分割。
* `val.txt`，验证数据列表，中间用\t（Tab水平制表符）进行分割。
* `labels.txt`，label的集合，表明训练数据多少标签。
* `data_pattern`参数，所有数据存在的目录的公共前缀，如果`train.txt`,`val.txt`文件里面存放的是绝对路径，`data_pattern`设置为None即可。

In [ ]:
import os
from collections import namedtuple
from onekey_algo.classification3d.run_classification3d import main as clf_main3d
from onekey_algo import get_param_in_cwd

# 设置参数
roi_size = [64, 64, 48]
vit_settings = {
        'image_size': roi_size[0],  # image size
        'frames': roi_size[-1],  # number of frames
        'image_patch_size': 16,  # image patch size
        'frame_patch_size': 2,  # frame patch size
        'dim': 1024,
        'depth': 6,
        'heads': 8,
        'mlp_dim': 2048,
        'dropout': 0.1,
        'emb_dropout': 0.1}
# 训练文件
train = get_param_in_cwd('train_file')
val = get_param_in_cwd('val_file')
data_pattern = get_param_in_cwd('data_pattern')
labels_file = r'labels.txt'

for model_name in get_param_in_cwd('model_names'):
    params = dict(train=train,
                  val=val,
                  labels_file=labels_file,
                  data_pattern=data_pattern,
                  j=0,
                  max2use=None,
                  normalize_method='imagenet',
                  model_name=model_name,
                  gpus=[0],
                  roi_size=roi_size,
                  batch_size=4,
                  epochs=get_param_in_cwd('epoch'),
                  init_lr=0.001,
                  optimizer='adam',
                  retrain=None,
                  model_root=get_param_in_cwd('model_root'),
                  val_interval=1,
                  save_per_epoch=False,
                  iters_verbose=10,
                  model_config={'groups': 1, 'blocks_args': 'efficientnet-b4'},
                  vit_settings=vit_settings,
                  pretrained=True)
    # 训练模型
    Args = namedtuple("Args", params)
    clf_main3d(Args(**params))